# Desafio 3

Neste desafio, iremos praticar nossos conhecimentos sobre distribuições de probabilidade. Para isso,
dividiremos este desafio em duas partes:
    
1. A primeira parte contará com 3 questões sobre um *data set* artificial com dados de uma amostra normal e
    uma binomial.
2. A segunda parte será sobre a análise da distribuição de uma variável do _data set_ [Pulsar Star](https://archive.ics.uci.edu/ml/datasets/HTRU2), contendo 2 questões.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

## Parte 1

### _Setup_ da parte 1

In [3]:
np.random.seed(42)
    
dataframe = pd.DataFrame({"normal": sct.norm.rvs(20, 4, size=10000),
                     "binomial": sct.binom.rvs(100, 0.2, size=10000)})

## Inicie sua análise a partir da parte 1 a partir daqui

In [4]:
dataframe.describe()

normal      binomial
count  10000.000000  10000.000000
mean      19.991456     20.097000
std        4.013850      4.041191
min        4.310399      6.000000
25%       17.309638     17.000000
50%       19.989620     20.000000
75%       22.684324     23.000000
max       35.704951     36.000000

## Questão 1

Qual a diferença entre os quartis (Q1, Q2 e Q3) das variáveis `normal` e `binomial` de `dataframe`? Responda como uma tupla de três elementos arredondados para três casas decimais.

Em outra palavras, sejam `q1_norm`, `q2_norm` e `q3_norm` os quantis da variável `normal` e `q1_binom`, `q2_binom` e `q3_binom` os quantis da variável `binom`, qual a diferença `(q1_norm - q1 binom, q2_norm - q2_binom, q3_norm - q3_binom)`?

In [5]:
def q1():
    quartis_normal = np.quantile(dataframe['normal'],[0.25,0.5,0.75])
    quartis_bin = np.quantile(dataframe['binomial'],[0.25,0.5,0.75])
    return tuple((quartis_normal - quartis_bin).round(3))

Para refletir:

* Você esperava valores dessa magnitude?

* Você é capaz de explicar como distribuições aparentemente tão diferentes (discreta e contínua, por exemplo) conseguem dar esses valores?

## Questão 2

Considere o intervalo $[\bar{x} - s, \bar{x} + s]$, onde $\bar{x}$ é a média amostral e $s$ é o desvio padrão. Qual a probabilidade nesse intervalo, calculada pela função de distribuição acumulada empírica (CDF empírica) da variável `normal`? Responda como uma único escalar arredondado para três casas decimais.

In [6]:
def q2():
    fnecdf=ECDF(dataframe['normal'])
    valor = fnecdf([dataframe['normal'].mean() - dataframe['normal'].std(), dataframe['normal'].mean() + dataframe['normal'].std()])[1]-fnecdf([dataframe['normal'].mean() - dataframe['normal'].std(), dataframe['normal'].mean() + dataframe['normal'].std()])[0]
    return float(valor.round(3))

Para refletir:

* Esse valor se aproxima do esperado teórico?
* Experimente também para os intervalos $[\bar{x} - 2s, \bar{x} + 2s]$ e $[\bar{x} - 3s, \bar{x} + 3s]$.

## Questão 3

Qual é a diferença entre as médias e as variâncias das variáveis `binomial` e `normal`? Responda como uma tupla de dois elementos arredondados para três casas decimais.

Em outras palavras, sejam `m_binom` e `v_binom` a média e a variância da variável `binomial`, e `m_norm` e `v_norm` a média e a variância da variável `normal`. Quais as diferenças `(m_binom - m_norm, v_binom - v_norm)`?

In [7]:
def q3():
    dif_means = dataframe['binomial'].mean() - dataframe['normal'].mean()
    dif_vars = dataframe['binomial'].var() - dataframe['normal'].var()
    resposta = np.array([dif_means,dif_vars])
    return tuple(resposta.round(3))

Para refletir:

* Você esperava valore dessa magnitude?
* Qual o efeito de aumentar ou diminuir $n$ (atualmente 100) na distribuição da variável `binomial`?

## Parte 2

### _Setup_ da parte 2

In [8]:
stars = pd.read_csv("pulsar_stars.csv")

stars.rename({old_name: new_name
              for (old_name, new_name)
              in zip(stars.columns,
                     ["mean_profile", "sd_profile", "kurt_profile", "skew_profile", "mean_curve", "sd_curve", "kurt_curve", "skew_curve", "target"])
             },
             axis=1, inplace=True)

stars.loc[:, "target"] = stars.target.astype(bool)

## Inicie sua análise da parte 2 a partir daqui

In [9]:
stars.head(3)

mean_profile  sd_profile  kurt_profile  skew_profile  mean_curve  \
0    102.507812   58.882430      0.465318     -0.515088    1.677258   
1    103.015625   39.341649      0.323328      1.051164    3.121237   
2    136.750000   57.178449     -0.068415     -0.636238    3.642977   

    sd_curve  kurt_curve  skew_curve  target  
0  14.860146   10.576487  127.393580   False  
1  21.744669    7.735822   63.171909   False  
2  20.959280    6.896499   53.593661   False

In [10]:
false_pulsar_mean_profile_standardized = stars[stars['target']==0]['mean_profile']
false_pulsar_mean_profile_standardized=(false_pulsar_mean_profile_standardized-false_pulsar_mean_profile_standardized.mean())/false_pulsar_mean_profile_standardized.std()
false_pulsar_mean_profile_standardized.head(3)

0   -0.804181
1   -0.775123
2    1.155263
Name: mean_profile, dtype: float64

In [11]:
quartis_pulsar = np.quantile(false_pulsar_mean_profile_standardized,[0.25,0.5,0.75])
quartis_norm = sct.norm.ppf([0.25,0.5,0.75])

## Questão 4

Considerando a variável `mean_profile` de `stars`:

1. Filtre apenas os valores de `mean_profile` onde `target == 0` (ou seja, onde a estrela não é um pulsar).
2. Padronize a variável `mean_profile` filtrada anteriormente para ter média 0 e variância 1.

Chamaremos a variável resultante de `false_pulsar_mean_profile_standardized`.

Encontre os quantis teóricos para uma distribuição normal de média 0 e variância 1 para 0.80, 0.90 e 0.95 através da função `norm.ppf()` disponível em `scipy.stats`.

Quais as probabilidade associadas a esses quantis utilizando a CDF empírica da variável `false_pulsar_mean_profile_standardized`? Responda como uma tupla de três elementos arredondados para três casas decimais.

In [12]:
def q4():
    pulsarecdf=ECDF(false_pulsar_mean_profile_standardized)
    return tuple(pulsarecdf(sct.norm.ppf([0.8,0.9,0.95])).round(3))

Para refletir:

* Os valores encontrados fazem sentido?
* O que isso pode dizer sobre a distribuição da variável `false_pulsar_mean_profile_standardized`?

## Questão 5

Qual a diferença entre os quantis Q1, Q2 e Q3 de `false_pulsar_mean_profile_standardized` e os mesmos quantis teóricos de uma distribuição normal de média 0 e variância 1? Responda como uma tupla de três elementos arredondados para três casas decimais.

In [13]:
def q5():
    difq = quartis_pulsar - quartis_norm
    return tuple(difq.round(3))

Para refletir:

* Os valores encontrados fazem sentido?
* O que isso pode dizer sobre a distribuição da variável `false_pulsar_mean_profile_standardized`?
* Curiosidade: alguns testes de hipóteses sobre normalidade dos dados utilizam essa mesma abordagem.